# Vítej!

Tento dokument obsahuje zdrojové kódy animací ke třetí sérii seriálu KSP. Před spouštěním opět nezapomeň Manim importovat spuštěním následujícího řádku!

In [ ]:
from manim import *

## `save` a `restore`

In [ ]:
%%manim -v WARNING -qh SaveAndRestoreExample


class SaveAndRestoreExample(Scene):
    def construct(self):
        square = Square()

        # uložení toho, jak aktualně čtverec vypadá
        square.save_state()

        self.play(Write(square))

        self.play(square.animate.set_fill(WHITE, 1))
        self.play(square.animate.scale(1.5).rotate(PI / 4))
        self.play(square.animate.set_color(BLUE))

        # navrácení do původního stavu
        self.play(square.animate.restore())

        # nová animace!
        self.play(Unwrite(square))

## Grafy

In [ ]:
%%manim -v WARNING -qh GraphExample


class GraphExample(Scene):
    def construct(self):
        # graf očekává vrcholy a hrany v tomto tvaru
        vertices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
        edges = [
            (1, 2),
            (2, 3),
            (3, 4),
            (2, 4),
            (2, 5),
            (6, 5),
            (1, 7),
            (5, 7),
            (2, 8),
            (1, 9),
            (10, 8),
            (5, 11),
        ]

        # layout_config používáme k tomu, aby byl algoritmus nastavující pozice vrcholů
        # deterministický (ten defaultní potřebuje seed)
        g = Graph(vertices, edges, layout_config={"seed": 0}).scale(1.6)

        self.play(Write(g))

        # graf obsahuje updatery, aby se hrany posouvaly s vrcholy
        self.play(g.vertices[6].animate.shift((LEFT + DOWN) * 0.5))

        self.play(g.animate.shift(LEFT * 3))

        # grafy mohou rovněž obsahovat labely a mohou být uspořádány do jiných layoutů
        # (pro ukázku všech viz link na dokumentaci třídy v seriálu)
        h = Graph(vertices, edges, labels=True, layout="circular").shift(RIGHT * 3)

        self.play(Write(h))

        # pokud chceme větší vrcholy i bez labelů, tak si je musíme obstarat manuálně
        self.play(*[g.vertices[v].animate.scale(2.15) for v in g.vertices])

        # obarvíme vrchol 5 a jemu odpovídající hrany (může se hodit v jedné z úloh :)
        v = 5
        self.play(
            Flash(g.vertices[v], color=RED, flash_radius=0.5),
            g.vertices[v].animate.set_color(RED),
            *[g.edges[e].animate.set_color(RED) for e in g.edges if v in e],
        )

In [ ]:
%%manim -v WARNING -qh GraphGenerationExample

from random import *
import networkx as nx


class GraphGenerationExample(Scene):
    def construct(self):
        seed(0xDEADBEEF)

        n = 12  # počet vrcholů
        p = 3 / n  # pravděpodobnost, že mezi dvěma vrcholy je hrana

        # generujeme, dokud nemáme spojitý graf
        graph = None
        while graph is None or not nx.is_connected(graph):
            graph = nx.generators.random_graphs.gnp_random_graph(n, p)

        g = (
            Graph(graph.nodes, graph.edges, layout_config={"seed": 0})
            .scale(2.2)
            .rotate(-PI / 2)
        )

        self.play(Write(g))

## Kamera

In [ ]:
%%manim -v WARNING -qh MovingCameraExample


class MovingCameraExample(MovingCameraScene):
    def construct(self):
        square = Square()

        self.play(Write(square))

        # uložíme si stav kamery, protože s ní následně budeme pracovat
        self.camera.frame.save_state()

        # zoomneme tak, aby čtverec zaplňoval celý obraz (+ drobná mezera)
        self.play(self.camera.frame.animate.set_height(square.height * 1.5))

        circle = Circle().next_to(square, LEFT)

        # posunutí kamery k novému objektu (kruhu)
        self.play(
            AnimationGroup(
                self.camera.frame.animate.move_to(circle),
                Write(circle),
                lag_ratio=0.5,
            )
        )

        self.wait(0.5)

        # trošku odzoomujeme (zvětšení bude zabírat více scény)
        self.play(self.camera.frame.animate.scale(1.3))

        triangle = Triangle().next_to(square, RIGHT)

        # posunutí kamery k novému objektu (trojúhelníku)
        self.play(
            AnimationGroup(
                self.camera.frame.animate.move_to(triangle),
                Write(triangle),
                lag_ratio=0.5,
            )
        )

        self.wait(0.5)

        # navrácení kamery do původního stavu
        self.play(self.camera.frame.animate.restore())

In [ ]:
%%manim -v WARNING -qh MovingCameraUpdaterExample

from random import seed, uniform


class MovingCameraUpdaterExample(MovingCameraScene):
    def construct(self):
        seed(0xDEADBEEF)

        n = 11 ** 2

        circles = VGroup(
            *[
                Circle(radius=0.1)
                .scale(uniform(0.5, 2))
                .shift(UP * uniform(-3, 3) + RIGHT * uniform(-5, 5))
                .set_color(WHITE)
                for _ in range(n)
            ]
        )

        # kamerou budeme pozorovat kruh, který je v půlce
        target = circles[n // 2]

        def update_curve(camera):
            """Updater, který udržuje kameru nad cílem."""
            camera.move_to(target.get_center())

        self.camera.frame.add_updater(update_curve)

        # POZOR!
        # updatery fungují pouze na věci přidané na scénu
        # self.camera.frame na scéně nejprve není, je ho potřeba přidat
        self.add(self.camera.frame)

        self.play(FadeIn(circles))

        # animace pozicování kružnic a postupné Zoomování
        scale_factor = 0.7

        self.play(
            circles.animate.arrange_in_grid().set_color(RED),
            self.camera.frame.animate.scale(scale_factor),
            run_time=1.5,
        )

        self.play(
            circles.animate.arrange_in_grid(rows=5).set_color(GREEN),
            self.camera.frame.animate.scale(scale_factor),
            run_time=1.5,
        )

        self.play(
            circles.animate.arrange_in_grid(cols=14).set_color(BLUE),
            self.camera.frame.animate.scale(scale_factor),
            run_time=1.5,
        )

In [ ]:
%%manim -v WARNING -qh BackgroundColorExample


class BackgroundColorExample(MovingCameraScene):
    def construct(self):
        self.camera.background_color = WHITE
        self.camera.frame.scale(0.6)

        square = Square(color=BLACK)

        self.play(Write(square))

        circle = Circle(color=BLACK).next_to(square, LEFT)
        triangle = Triangle(color=BLACK).next_to(square, RIGHT)

        self.play(FadeIn(triangle, shift=RIGHT * 0.2), FadeIn(circle, shift=LEFT * 0.2))

## Rate funkce

In [ ]:
%%manim -v WARNING -qh RateFunctionsExample

# Kód pochází z Manimové dokumentace (modulo drobné úpravy)
# https://docs.manim.community/en/stable/reference/manim.utils.rate_functions.html


class RateFunctionsExample(Scene):
    def construct(self):
        line1 = Line(3 * LEFT, RIGHT).set_color(RED)
        line2 = Line(3 * LEFT, RIGHT).set_color(GREEN)
        line3 = Line(3 * LEFT, RIGHT).set_color(BLUE)
        line4 = Line(3 * LEFT, RIGHT).set_color(ORANGE)

        lines = VGroup(line1, line2, line3, line4).arrange(DOWN, buff=0.8).move_to(LEFT * 2)

        dot1 = Dot().move_to(line1.get_start())
        dot2 = Dot().move_to(line2.get_start())
        dot3 = Dot().move_to(line3.get_start())
        dot4 = Dot().move_to(line4.get_start())

        dots = VGroup(dot1, dot2, dot3, dot4)

        # pozor na podtržítka při psaní TeXu
        label1 = Tex("smooth (default)").next_to(line1, RIGHT, buff=0.5)
        label2 = Tex("linear").next_to(line2, RIGHT, buff=0.5)
        label3 = Tex("there\_and\_back").next_to(line3, RIGHT, buff=0.5)
        label4 = Tex("rush\_into").next_to(line4, RIGHT, buff=0.5)

        labels = VGroup(label1, label2, label3, label4)

        self.play(Write(lines), FadeIn(dots), FadeIn(labels))

        # použití s animate syntaxem
        self.play(
            dot1.animate(rate_func=smooth).shift(RIGHT * 4),
            dot2.animate(rate_func=linear).shift(RIGHT * 4),
            dot3.animate(rate_func=there_and_back).shift(RIGHT * 4),
            dot4.animate(rate_func=rush_into).shift(RIGHT * 4),
            run_time=3,
        )
        
        self.play(FadeOut(lines), FadeOut(dots))
        
        # použití s normálními animacemi
        self.play(
            Write(line1, rate_func=smooth),
            Write(line2, rate_func=linear),
            Write(line3, rate_func=there_and_back),
            Write(line4, rate_func=rush_into),
            run_time=3,
        )

In [ ]:
%%manim -v WARNING -qh RateFunctionList

# Kód pochází z Manimové dokumentace (modulo drobné úpravy)
# https://docs.manim.community/en/stable/reference/manim.utils.rate_functions.html


class RateFunctionList(Scene):
    def construct(self):
        graphs = VGroup()
        for k, v in rate_functions.__dict__.items():
            if "function" in str(v):
                try:
                    rate_func = v
                    plot = (
                        ParametricFunction(
                            lambda x: [x, rate_func(x), 0],
                            t_range=[0, 1, .01],
                            use_smoothing=False,
                            color=YELLOW,
                        )
                        .stretch_to_fit_width(1.5)
                        .stretch_to_fit_height(1)
                    )
                    plot_bg = SurroundingRectangle(plot).set_color(WHITE)
                    plot_title = (
                        Text(rate_func.__name__, weight=BOLD)
                        .scale(0.5)
                        .next_to(plot_bg, UP, buff=0.1)
                    )
                    graphs.add(VGroup(plot_bg, plot, plot_title))
                except:  # některé křivky jsou parametrické (a některé rozbité)
                    pass
        graphs.arrange_in_grid(cols=8)
        graphs.height = config.frame_height
        graphs.width = config.frame_width
        graphs.move_to(ORIGIN).scale(0.9)
        
        text = Tex(f"Manim rate functions (v0.13.1)").scale(1.4).next_to(graphs, UP, buff=0.5)
        
        self.add(VGroup(graphs, text).move_to(ORIGIN))

## Krokování

In [ ]:
%%manim -v WARNING -qh --save_sections NextSectionExample

class NextSectionExample(Scene):
    def construct(self):
        shapes = VGroup(Circle(), Square(), Triangle()).arrange()
                        
        # jméno sekce je nepovinné a nemusí být unikátní
        # skip_animations animování sekce přeskočí
        self.next_section("vykreslení kruhu", skip_animations=True)
                        
        self.play(Write(shapes[0]))
                        
        self.next_section("vykreslení čtverce")
                        
        self.play(Write(shapes[1]))
                        
        self.next_section("vykreslení trojúhelníku")
                        
        self.play(Write(shapes[2]))

## Kostry úloh

### Grafový algoritmus [5b]

In [ ]:
%%manim -v WARNING -qh GraphAlgorithm


class GraphAlgorithm(Scene):
    def construct(self):
        pass

### Fibonacciho posloupnost [5b]

In [ ]:
%%manim -v WARNING -qh --disable_caching FibonacciSequence


class FibonacciSequence(MovingCameraScene):
    def construct(self):
        pass

In [ ]:
%%manim -v WARNING -qh --disable_caching TracePathExample

            
class TracePathExample(Scene):
    def construct(self):
        dot = Dot().shift(LEFT)
            
        self.play(Write(dot))
        
        # objekt TracedPath přijímá funkci, které se opakovaně ptá na pozici objektu, který sleduje
        # proto používáme dot.get_center, která vrací aktuální pozici tečky kterou tracujeme
        path = TracedPath(dot.get_center)
        
        # nesmíme zapomenout cestu přidat do scény, aby byla vykreslována!
        self.add(path)
        
        self.play(Rotate(dot, about_point=ORIGIN))
        
        self.play(dot.animate.shift(UP))
        self.play(dot.animate.shift(LEFT * 2))
        self.play(dot.animate.shift(DOWN))
        
        # pro zkončení tracování můžeme použít clear_updaters
        path.clear_updaters()
        
        self.play(dot.animate.shift(RIGHT * 2))

### Langtonův mravenec [5b]

In [ ]:
%%manim -v WARNING -qh LangtonAnt


class LangtonAnt(MovingCameraScene):
    def construct(self):
        pass

In [ ]:
%%manim -v WARNING -qh SVGExample


class SVGExample(Scene):
    def construct(self):
        image = SVGMobject("ant.svg")

        self.play(Write(image))

        self.play(image.animate.set_color(RED).scale(1.75))

        self.play(Rotate(image, TAU))  # tau = 2 pi

        self.play(FadeOut(image))